## MDP
[Latex symbols](https://oeis.org/wiki/List_of_LaTeX_mathematical_symbols)
[Solving an MDP with Q-Learning from scratch ](https://medium.com/@curiousily/solving-an-mdp-with-q-learning-from-scratch-deep-reinforcement-learning-for-hackers-part-1-45d1d360c120)
Rewards:  
$R_t = r_t + r_{t+1} + ... + r_n$

Discounted reward:
$R_t = R_t + \gamma r_{t+1} + ... + \gamma^{n-t}r_n = r_t + \gamma R_{t+1}$

Агент должен выбирать то действие, которое позволит ему максимизировать будущий ревард на каждом шаге.

**Value function:**

Позволяет оценить насколько хорошим является состояние $s$. Позволяет оценить следующее: неважно в каком состоянии ты сейчас, но если ты перейдёшь в состояние $s$, то твой полный ревард будет $x$. (если ты пойдёшь в s и будешь следовать полиси $\pi$.

$V^\pi(s) = E (\sum\limits_{t \geq 0} \gamma^t r_t) \ \ \forall s \in S$
Where: $s$ - state, $\pi$ - policy, $V$ - value function.

**Optimal value Function**

Возвращает наибольшее значение для всех состояний.

$V^*(s) = \max\limits_{\pi} V^\pi (s) \ \ \forall s \in S$


**Q function**

Данная функция позволяет оценить полный ревард по 

Соотношение между функциями $V^∗$ и $Q^∗$


$V^*(s) = \max\limits_{a} Q^*(s) \ \ \forall s \in S$

That is, the maximum expected total reward when starting at $s$ is the maximum of $Q^∗(s, a)$ over all possible actions.


Using Q∗(s, a) we can extract the optimal policy π∗ by choosing the action aa that gives maximum reward Q∗(s, a) for state s. We have:

$\pi^*(s) = \arg \max\limits_{a} Q^* (s) \ \ \forall s \in S$

$Q(s,a) = r + \gamma \max\limits_{a'} Q (s',a')$

This equation, known as the Bellman equation, tells us that the maximum future reward is the reward the agent received for entering the current state s plus the maximum future reward for the next state s′. The gist of Q-learning is that we can iteratively approximate Q∗ using the Bellman equation described above.

$Q_{t+1}(s_t,a_t) = Q_{t}(s_t,a_t) + \alpha(r_{t+1} + \gamma \max\limits_{a} Q (s_{t+1},a) -  Q (s_{t+1},a) ) $

In [43]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [3]:
ZOMBIE = "z"
CAR = "c"
ICE_CREAM = "i"
EMPTY = "*"

grid = [
    [ICE_CREAM, EMPTY],
    [ZOMBIE, CAR]
]

for row in grid:
    print(' '.join(row))


i *
z c


In [19]:
class State:
    def __init__(self, grid, car_pos):
        self.grid = grid
        self.car_pos = car_pos
    def __eq__(self, other):
        return isinstance(other, State) and self.grid == other.grid and self.car_pos == other.car_pos
    
    def __hash__(self):
        return hash(str(self.grid) + str(self.car_pos))
    def __str(self):
        return f"State(grid={self.grid}, car_pos={self.car_pos})"
    

In [20]:
#actions
UP = 0
DOWN = 1
LEFT = 2
RIGHT = 3
ACTIONS = [UP, DOWN, LEFT, RIGHT]

In [21]:
start_state = State(grid=grid, car_pos=[1,1])

In [29]:
from copy import deepcopy

def act(state, action):
    def new_car_pos(state, action):
        p = deepcopy(state.car_pos)
        if action == UP:
            p[0] = max(0, p[0]-1)
        elif action == DOWN:
            p[0] == min(len(state.grid) - 1, p[0] + 1)
        elif action == LEFT:
            p[1] = max(0, p[1] - 1)
        elif action == RIGHT:
            p[1] = min(len(state.grid[0]) - 1, p[1] + 1)
        else:
            raise ValueError(f"Unknown action {action}")
        
        return p
    
    p = new_car_pos(state, action)
    grid_item = state.grid[p[0]][p[1]]
    
    new_grid = deepcopy(state.grid)
    
    if grid_item == ZOMBIE:
        reward = -100
        is_done = True
        new_grid[p[0]][p[1]] += CAR
    elif grid_item == ICE_CREAM:
        reward = 1000
        is_done = True
        new_grid[p[0]][p[1]] += CAR
    elif grid_item == EMPTY:
        reward = -1
        is_done = False
        old = state.car_pos
        new_grid[old[0]][old[1]] = EMPTY
        new_grid[p[0]][p[1]] = CAR
    elif grid_item == CAR:
        reward = -1
        is_done = False
    else:
        raise ValueError(f"Unknown grid item {grid_item}")
    
    return State(grid=new_grid, car_pos = p), reward, is_done


In [23]:
import numpy as np
import random

random.seed(42)

N_STATES = 4
N_EPISODES = 20

MAX_EPISODE_STEPS = 100

MIN_ALPHA = 0.02

alphas = np.linspace(1.0, MIN_ALPHA, N_EPISODES)
gamma = 1.0
eps = 0.2

q_table = dict()


In [24]:
def q(state, action = None):
    if state not in q_table:
        q_table[state] = np.zeros(len(ACTIONS))
    
    if action is None:
        return q_table[state]
    
    return q_table[state][action]

In [25]:
def choose_action(state):
    if random.uniform(0,1) < eps:
        return random.choice(ACTIONS)
    else:
        return np.argmax(q(state))

In [45]:
for e in range(N_EPISODES):
    state = start_state
    total_reward = 0
    alpha = alphas[e]
    
    for _ in range(MAX_EPISODE_STEPS):
        action = choose_action(state)
        next_state, reward, done = act(state, action)
        total_reward += reward
        
        q(state)[action] = q(state, action ) \
        + alpha * (reward + gamma * np.max(q(next_state) - q(state, action)))
        state = next_state
        if done:
            break
    print(f"Episode {e+1}: total reward -> {total_reward}")

Episode 1: total reward -> 999
Episode 2: total reward -> -100
Episode 3: total reward -> 999
Episode 4: total reward -> 999
Episode 5: total reward -> 999
Episode 6: total reward -> 999
Episode 7: total reward -> 999
Episode 8: total reward -> 999
Episode 9: total reward -> 999
Episode 10: total reward -> 998
Episode 11: total reward -> 999
Episode 12: total reward -> 999
Episode 13: total reward -> 998
Episode 14: total reward -> 999
Episode 15: total reward -> 999
Episode 16: total reward -> 998
Episode 17: total reward -> 999
Episode 18: total reward -> 999
Episode 19: total reward -> 999
Episode 20: total reward -> 998


In [46]:
r = q(start_state)
print(f"up={r[UP]}, down={r[DOWN]}, left={r[LEFT]}, right={r[RIGHT]}")

up=999.0, down=997.971991504285, left=-100.0, right=998.0


In [47]:
new_state, rewrd, done = act(start_state, UP)

In [48]:
r = q(new_state)
print(f"up={r[UP]}, down={r[DOWN]}, left={r[LEFT]}, right={r[RIGHT]}")

up=999.0, down=998.9988339927265, left=1000.0, right=999.0
